In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [2]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.Map.Strict as M
import Data.Map.Strict ((!))

import Control.Monad (when)
import Control.Monad.State.Lazy
import Control.Monad.Reader
import Control.Monad.Writer

import Data.Numbers.Primes

<interactive>:1:1: error:
    Failed to load interface for ‘Data.Numbers.Primes’
    Use -v to see a list of the files searched for.

In [3]:
data Location = Literal Integer | Register Char deriving (Show, Eq)
data Instruction =   Set Location Location 
                   | Sub Location Location 
                   | Mul Location Location
                   | Jnz Location Location
                   deriving (Show, Eq)

data Machine = Machine { registers :: M.Map Char Integer
                       , pc :: Int
                       } 
               deriving (Show, Eq)

type ProgrammedMachine = WriterT [String] (ReaderT [Instruction] (State Machine)) ()

In [4]:
emptyMachine = Machine {registers = M.empty, pc = 0}

In [5]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme  = L.lexeme sc

integer       = lexeme L.integer
signedInteger = L.signed sc integer

symbol = L.symbol sc

-- reg :: Parser String
-- reg = id <$> some letterChar

reg = lexeme (some letterChar)

location = (Literal <$> signedInteger) <|> register
register = (Register . head) <$> reg

instructionsP = instructionP `sepBy` space
instructionP = choice [setP, subP, mulP, jnzP]

setP = Set <$> (try (symbol "set") *> register) <*> location
subP = Sub <$> (try (symbol "sub") *> register) <*> location
mulP = Mul <$> (try (symbol "mul") *> register) <*> location
jnzP = Jnz <$> (try (symbol "jnz") *> location) <*> location

successfulParse :: Text -> [Instruction]
successfulParse input = 
        case parse instructionsP "input" input of
                Left  _error -> [] -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right instructions  -> instructions

In [6]:
sample = T.pack "set a 1\nsub a 2\nmul a a\njnz a 5"

In [7]:
sampleInstructions = successfulParse sample
sampleInstructions

[Set (Register 'a') (Literal 1),Sub (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Jnz (Register 'a') (Literal 5)]

In [8]:
isMul :: Instruction -> Bool
isMul (Mul _ _ ) = True
isMul _ = False

In [9]:
isJnz :: Instruction -> Bool
isJnz (Jnz _ _ ) = True
isJnz _ = False

In [10]:
evaluate :: Machine -> Location -> Integer
evaluate _ (Literal i)  = i
evaluate m (Register r) = M.findWithDefault 0 r (registers m)

In [11]:
applyInstruction :: Instruction -> Machine -> Machine

applyInstruction (Set (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          y = evaluate m b
          reg' = M.insert a y $ registers m

applyInstruction (Sub (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x - y) $ registers m

applyInstruction (Mul (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x * y) $ registers m

applyInstruction (Jnz a b) m = m {pc = pc'}
    where x = evaluate m a
          y = evaluate m b
          pc' = if x /= 0 then pc m + (fromIntegral y) else pc m + 1

In [28]:
executeInstructionPeep :: ProgrammedMachine
executeInstructionPeep =
    do  instrs <- ask
        m <- get
        let sample1 = take (length sample1Target) $ drop (pc m) $ instrs
        if sample1 == sample1Target
            -- then trace ("Peeping 1 " ++ (show m) ++ " to " ++ (show m1)) m1
            then do let reg1 = M.union (M.fromList [ ('d', 2), ('e', evaluate m (Register 'b'))
                                                   , ('f', 0), ('g', 0)
                                                   ]) 
                                       (registers m)
                    let m1 = m {registers = reg1, pc = pc m + (length sample1)}
                    put m1
            else executeInstruction
    where 
--           sample1 = take (length sample1Target) $ drop (pc m) $ instrs
          sample1Target = [ Set (Register 'b') (Literal 4)
                          , Set (Register 'f') (Literal 1)
                          , Set (Register 'd') (Literal 2)
                          , Set (Register 'e') (Literal 2)
                          , Set (Register 'g') (Register 'd')
                          , Mul (Register 'g') (Register 'e')
                          , Sub (Register 'g') (Register 'b')
                          , Jnz (Register 'g') (Literal 2)
                          , Set (Register 'f') (Literal 0)
                          , Sub (Register 'e') (Literal (-1))
                          , Set (Register 'g') (Register 'e')
                          , Sub (Register 'g') (Register 'b')
                          , Jnz (Register 'g') (Literal (-8))
                          ]
--           reg1 = M.union (M.fromList [ ('d', 2), ('e', evaluate m (Register 'b'))
--                                      , ('f', 0), ('g', 0)
--                                      ]) 
--                          (registers m)
--           m1 = m {registers = reg1, pc = pc m + (length sample1)}
          

In [29]:
executeInstruction :: ProgrammedMachine
executeInstruction =
    do  instrs <- ask
        m <- get
        let instr = instrs!!(pc m)
--         tell [("pc = " ++ show (pc m))]
        put (applyInstruction instr m)

In [30]:
executeInstructions = 
    do  instrs <- ask
        m <- get
        when (pc m >= 0 && pc m < length instrs)
            $
            do when (isMul $ instrs !! pc m) (tell ["mul"])
               when (isJnz $ instrs !! pc m) (tell [show m])
--                executeInstructionPeep
               executeInstruction
               executeInstructions

In [39]:
executeInstructionsPeep = 
    do  instrs <- ask
        m <- get
        when (pc m >= 0 && pc m < length instrs)
            $
            do -- when (isMul $ instrs !! pc m) (tell ["mul"])
               -- when (isJnz $ instrs !! pc m) (tell [show m])
               executeInstructionPeep
               executeInstructionsPeep

In [40]:
runState (runReaderT (runWriterT executeInstructions) sampleInstructions ) emptyMachine

(((),["mul","Machine {registers = fromList [('a',1)], pc = 3}"]),Machine {registers = fromList [('a',1)], pc = 8})

In [41]:
runState (
    runReaderT (
        runWriterT executeInstructions
               ) 
        (take 7 sampleInstructions) 
         ) 
         emptyMachine

(((),["mul","Machine {registers = fromList [('a',1)], pc = 3}"]),Machine {registers = fromList [('a',1)], pc = 8})

In [42]:
sampleInstructions

[Set (Register 'a') (Literal 1),Sub (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Jnz (Register 'a') (Literal 5)]

In [43]:
part1 instructions = 
    runState (
        runReaderT (
            runWriterT executeInstructions
                   ) 
            instructions 
             ) 
             emptyMachine

In [44]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent23.txt"
        let instrs = successfulParse text
        let ((result, l), machinef) = part1 instrs
--         print $ head l
        print $ length $ filter (== "mul") l
--         print $ part2 instrs

In [45]:
main

6724

In [46]:
runTest instructions machine0 = 
    runState (
        runReaderT (
            runWriterT executeInstructions
                   ) 
            instructions 
             ) 
             machine0

Line 1: Eta reduce
Found:
runTest instructions machine0
  = runState
      (runReaderT (runWriterT executeInstructions) instructions)
      machine0
Why not:
runTest instructions
  = runState
      (runReaderT (runWriterT executeInstructions) instructions)

In [47]:
runTestPeep instructions machine0 = 
    runState (
        runReaderT (
            runWriterT executeInstructionsPeep
                   ) 
            instructions 
             ) 
             machine0

Line 1: Eta reduce
Found:
runTestPeep instructions machine0
  = runState
      (runReaderT (runWriterT executeInstructionsPeep) instructions)
      machine0
Why not:
runTestPeep instructions
  = runState
      (runReaderT (runWriterT executeInstructionsPeep) instructions)

In [48]:
peepTest1T = T.pack "set b 4\nset f 1\nset d 2\nset e 2\nset g d\nmul g e\nsub g b\njnz g 2\nset f 0\nsub e -1\nset g e\nsub g b\njnz g -8"
peepTest1 = successfulParse peepTest1T
peepTest1

[Set (Register 'b') (Literal 4),Set (Register 'f') (Literal 1),Set (Register 'd') (Literal 2),Set (Register 'e') (Literal 2),Set (Register 'g') (Register 'd'),Mul (Register 'g') (Register 'e'),Sub (Register 'g') (Register 'b'),Jnz (Register 'g') (Literal 2),Set (Register 'f') (Literal 0),Sub (Register 'e') (Literal (-1)),Set (Register 'g') (Register 'e'),Sub (Register 'g') (Register 'b'),Jnz (Register 'g') (Literal (-8))]

In [49]:
((v, t), m) = runTest peepTest1 emptyMachine
t

["mul","Machine {registers = fromList [('b',4),('d',2),('e',2),('f',1),('g',0)], pc = 7}","Machine {registers = fromList [('b',4),('d',2),('e',3),('f',0),('g',-1)], pc = 12}","mul","Machine {registers = fromList [('b',4),('d',2),('e',3),('f',0),('g',2)], pc = 7}","Machine {registers = fromList [('b',4),('d',2),('e',4),('f',0),('g',0)], pc = 12}"]

In [50]:
text <- TIO.readFile "../../data/advent23.txt"
let fullInstrs = successfulParse text

In [51]:
fullInstrs

[Set (Register 'b') (Literal 84),Set (Register 'c') (Register 'b'),Jnz (Register 'a') (Literal 2),Jnz (Literal 1) (Literal 5),Mul (Register 'b') (Literal 100),Sub (Register 'b') (Literal (-100000)),Set (Register 'c') (Register 'b'),Sub (Register 'c') (Literal (-17000)),Set (Register 'f') (Literal 1),Set (Register 'd') (Literal 2),Set (Register 'e') (Literal 2),Set (Register 'g') (Register 'd'),Mul (Register 'g') (Register 'e'),Sub (Register 'g') (Register 'b'),Jnz (Register 'g') (Literal 2),Set (Register 'f') (Literal 0),Sub (Register 'e') (Literal (-1)),Set (Register 'g') (Register 'e'),Sub (Register 'g') (Register 'b'),Jnz (Register 'g') (Literal (-8)),Sub (Register 'd') (Literal (-1)),Set (Register 'g') (Register 'd'),Sub (Register 'g') (Register 'b'),Jnz (Register 'g') (Literal (-13)),Jnz (Register 'f') (Literal 2),Sub (Register 'h') (Literal (-1)),Set (Register 'g') (Register 'b'),Sub (Register 'g') (Register 'c'),Jnz (Register 'g') (Literal 2),Jnz (Literal 1) (Literal 3),Sub (Reg

In [52]:
((v, t), m) = runTest fullInstrs emptyMachine
m

Machine {registers = fromList [('b',84),('c',84),('d',84),('e',84),('f',0),('g',0),('h',1)], pc = 32}

In [53]:
((v, t), m) = runTestPeep fullInstrs emptyMachine
m

Machine {registers = fromList [('b',84),('c',84),('d',84),('e',84),('f',0),('g',0),('h',1)], pc = 32}

In [ ]:
((v, t), m) = runTestPeep fullInstrs (emptyMachine {registers = M.fromList [('a', 1)]})
m